In [83]:
from mlstudio_sdk.config import Config
import psycopg2
import psycopg2.extras

config = Config()

# def get_experiments_by(user_name) :
#     #establishing the connection
#     conn = psycopg2.connect(config.get_mlflow_tracking_auth_uri())

#     #Setting auto commit false
#     conn.autocommit = True

#     #Creating a cursor object using the cursor() method
#     cursor = conn.cursor()

#     sql = f"""
#     SELECT id FROM users
#     WHERE username='{user_name}'
#     """

#     #Doping EMPLOYEE table if already exists.
#     cursor.execute(sql)

#     # publisher_records = cursor.fetchall()
#     publisher_records = list(cursor.fetchone())
#     print(publisher_records)
    
#     #Commit your changes in the database
#     conn.commit()
    
#     #Closing the connection
#     conn.close()

def get_is_admin(user_name) :
    #establishing the connection
    conn = psycopg2.connect(config.get_mlflow_tracking_auth_uri())

    sql = f"""
    SELECT  is_admin
    FROM    users
    WHERE   username='{user_name}'
    """

    cursor = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
    cursor.execute(sql)
    r = cursor.fetchone()
    if not r : return False
    
    #Closing the connection
    conn.close()
    
    if 'is_admin' in r :
        return r['is_admin']
    return False

# 접근권한을 가지고 있는 experiment id 조회
def get_experiment_permissions(user_name) :
    #establishing the connection
    conn = psycopg2.connect(config.get_mlflow_tracking_auth_uri())
    
    sql = f"""
    SELECT  experiment_id,
        user_id,
        permission
    FROM   experiment_permissions experiments
    INNER JOIN (
        SELECT  id
        FROM    users
        WHERE   username='{user_name}'
    ) users
    ON experiments.user_id = users.id
    """

    cursor = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
    cursor.execute(sql)
    records = cursor.fetchall()
    
    rtn = []
    for r in records:
        rtn.append(dict(r))

    conn.close()
    
    return rtn

# 접근권한을 가지고 있는 experiment 전체정보 조회
def get_experiments_by_permission(user_name) :
    
    is_admin = get_is_admin(user_name)
    
    if not is_admin :
        permissions = get_experiment_permissions(user_name)

        if len(permissions) < 1:
            return None

        exps=[]
        for p in permissions:
            exps.append(p['experiment_id'])

        exps = ', '.join(map(str,exps))

        sql = f"""
            SELECT *
            FROM experiments
            WHERE experiment_id In ({exps});
            """
    else:
        sql = """
            SELECT *
            FROM experiments
            """
        
    conn = psycopg2.connect(config.get_mlflow_tracking_uri())
    cursor = conn.cursor(cursor_factory = psycopg2.extras.RealDictCursor)
    cursor.execute(sql)

    recoreds = cursor.fetchall()

    rtn = []
    for row in recoreds:
        rtn.append(dict(row))

    conn.close()

    return rtn

r = get_experiments_by_permission(user_name='test1')

print(r)

[{'experiment_id': 1, 'name': 'MLflow Quickstart', 'artifact_location': '/opt/mlflow/.mlruns/1', 'lifecycle_stage': 'active', 'creation_time': 1718526699581, 'last_update_time': 1718526699581}, {'experiment_id': 2, 'name': 'MLflow Quickstart2', 'artifact_location': '/opt/mlflow/.mlruns', 'lifecycle_stage': 'active', 'creation_time': 1718544083679, 'last_update_time': 1718544083679}]
